In [3]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from datasets import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [5]:
df = pd.read_csv("/content/Labeled_data.csv")

In [6]:
df.shape

(18095, 2)

In [7]:
df.head()

,processed_text,sentiment3
0,president trump please stop attacking biden li...,negative
1,bardella group grouped brake large family lzrt...,negative
2,terrorist de paepe chridtin chridtine king suf...,negative
3,terrorist serial criminal uou understand intel...,negative
4,others terrorist serial criminal,negative


In [8]:
label_map = {"negative":0, "neutral":1, "positive":2}
df['sentiment3'] =  df['sentiment3'].map(label_map)

In [9]:
df['sentiment3'].value_counts()

,count
sentiment3,
2,8426
1,5984
0,3685


In [10]:
df = df.dropna(subset=['processed_text'])

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example['processed_text'], padding="max_length", truncation=True,max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("sentiment3", "labels")
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/18073 [00:00<?, ? examples/s]

In [13]:
dataset

Dataset({
    features: ['processed_text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 18073
})

In [14]:
dataset = dataset.train_test_split(test_size=0.2)

In [15]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zaeemhassan2004 (zaeemhassan2004-zaeem-hassan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.603900
1000,0.545600
1500,0.526500
2000,0.503200
2500,0.500800
3000,0.478900
3500,0.440800


TrainOutput(global_step=3615, training_loss=0.5107092038229788, metrics={'train_runtime': 12353.3111, 'train_samples_per_second': 1.17, 'train_steps_per_second': 0.293, 'total_flos': 478811951221248.0, 'train_loss': 0.5107092038229788, 'epoch': 1.0})

In [16]:
trainer.evaluate()


{'eval_loss': 0.43869009613990784,
 'eval_runtime': 733.5727,
 'eval_samples_per_second': 4.928,
 'eval_steps_per_second': 0.308,
 'epoch': 1.0}

In [17]:
text = "I really love this video!"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
pred = outputs.logits.argmax(dim=1).item()

sentiment = {0: "negative", 1: "neutral", 2: "positive"}
print("Predicted sentiment:", sentiment[pred])


Predicted sentiment: positive


In [21]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")

('sentiment_model/tokenizer_config.json',
 'sentiment_model/special_tokens_map.json',
 'sentiment_model/vocab.txt',
 'sentiment_model/added_tokens.json')

In [22]:
!zip -r sentiment_model.zip sentiment_model


  adding: sentiment_model/ (stored 0%)
  adding: sentiment_model/config.json (deflated 49%)
  adding: sentiment_model/vocab.txt (deflated 53%)
  adding: sentiment_model/tokenizer_config.json (deflated 75%)
  adding: sentiment_model/model.safetensors (deflated 8%)
  adding: sentiment_model/special_tokens_map.json (deflated 42%)


In [23]:
from google.colab import files
files.download("sentiment_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>